In [2]:
import numpy as np

from helper_functions import get_covmats
from helper_functions import plot_kmeans_2D
from helper_functions import plot_em_2D
from helper_functions import match_labels
from helper_functions import calculate_distortions
from helper_functions import plot_distortions
from helper_functions import generate_and_plot_silhouette
from helper_functions import generate_and_plot_bic_em

from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn import random_projection
# from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
# import matplotlib.cm as cm
from scipy.stats import kurtosis
import time

NUM_CLASS = 2
folder = 'figures/breast_cancer/'

In [3]:
X_raw, y = load_breast_cancer(return_X_y=True)
X = scale(X_raw)

In [5]:
Ks = range(2, 20)

print('Generate elbow plots...')
distortions = np.zeros(len(Ks), dtype=np.float)
for i, K in enumerate(Ks):
    distortions[i] = calculate_distortions(K, X)
plot_distortions(Ks, distortions, 'Raw Data', folder+'kmeans_raw_elbow.png')
print('Done...')

Generate elbow plots...
Done...


In [6]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X, 'Raw Data', folder+'kmeans_raw_sil.png',
    xleft=-0.2, ncols=10
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [7]:
print('Plot k-means result...')
K = 2
km_est = KMeans(K, random_state=42)
y_hat = km_est.fit_predict(X)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total errors for Kmeans on raw: {}'.format(terrors))
plot_pca = PCA(2, random_state=42)
projs = plot_pca.fit_transform(X)
plot_kmeans_2D(projs[:, 0], projs[:, 1], y, y_hat,
               plot_pca.transform(km_est.cluster_centers_), NUM_CLASS, K,
               label_map, 'Raw data', folder+'kmeans_raw_result.png')
print('Done...')

print('Kmeans on raw data finished...')

Plot k-means result...
Total errors for Kmeans on raw: 51
Done...
Kmeans on raw data finished...


In [8]:
Ks = range(2, 20)

print()
print('Start using raw features for EM...')

print('Generate BIC bar plots...')
generate_and_plot_bic_em(Ks, X, 'Raw Data', folder+'em_raw_bic.png')
print('Done...')


Start using raw features for EM...
Generate BIC bar plots...
Done...


In [9]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X, 'Raw Data', folder+'em_raw_sil.png',
    xleft=-0.2, ncols=10, method='EM'
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [10]:
print('Plot EM result...')
K = 2
em_est = GaussianMixture(K, 'full', random_state=42)
y_hat = em_est.fit_predict(X)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total errors for EM on raw: {}'.format(terrors))
plot_pca = PCA(2, random_state=42)
projs = plot_pca.fit_transform(X)
covs = get_covmats(em_est.covariances_, X.shape[1], K, 'full')
covmats = np.matmul(
    np.matmul(plot_pca.components_, covs),
    plot_pca.components_.T
)
plot_em_2D(projs[:, 0], projs[:, 1], y, y_hat,
           plot_pca.transform(em_est.means_), covmats, NUM_CLASS, K,
           label_map, 'Raw Data', folder+'em_raw_result.png')
print('Done...')

print('EM on raw data finished...')

Plot EM result...
Total errors for EM on raw: 35
Done...
EM on raw data finished...


In [11]:
print()
print('Start using PCA for pre-processing...')

print('Plotting explained variance against K...')
K = X.shape[1]
pca = PCA(K, random_state=42)
X_pca = pca.fit_transform(X)

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2, 1, 1)
ax1.set_title('PCA variance explained')
ax1.set_xlabel('K')
ax1.set_ylabel('Percentage of variance')
ax1.plot(
    range(1, K+1), np.cumsum(pca.explained_variance_ratio_),
    'o-', color='blue'
)

ax2 = fig.add_subplot(2, 1, 2)
ax2.set_title('PCA eigenvalues')
ax2.set_xlabel('K')
ax2.set_ylabel('Eigenvalues')
ax2.plot(
    range(1, K+1), pca.explained_variance_,
    'o-', color='red'
)

plt.savefig(folder+'pca_variance_explained.png', format='png')
plt.close()
print('Done')


Start using PCA for pre-processing...
Plotting explained variance against K...
Done


In [12]:
K = np.sum(np.cumsum(pca.explained_variance_ratio_) <= 0.95)
print('Use {} components (explained var > 0.9)...'.format(K))

pca = PCA(K, random_state=42)
X_pca = pca.fit_transform(X)

Use 9 components (explained var > 0.9)...


In [13]:
Ks = range(2, 20)

print('Start using PCA features for K-means...')

print('Generate elbow plots...')
distortions = np.zeros(len(Ks), dtype=np.float)
for i, K in enumerate(Ks):
    distortions[i] = calculate_distortions(K, X_pca)
plot_distortions(Ks, distortions, 'PCA Data', folder+'kmeans_pca_elbow.png')
print('Done...')

Start using PCA features for K-means...
Generate elbow plots...
Done...


In [14]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_pca, 'PCA Data', folder+'kmeans_pca_sil.png',
    xleft=-0.2, ncols=10
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [15]:
print('Plot k-means result...')
K = 2
km_est = KMeans(K, random_state=42)
y_hat = km_est.fit_predict(X_pca)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for KMeans on PCA: {}'.format(terrors))
plot_kmeans_2D(X_pca[:, 0], X_pca[:, 1], y, y_hat,
               km_est.cluster_centers_[:, 0:2], NUM_CLASS, K,
               label_map, 'PCA data', folder+'kmeans_pca_result.png')
print('Done...')

print('Kmeans on PCA data finished...')

Plot k-means result...
Total error for KMeans on PCA: 51
Done...
Kmeans on PCA data finished...


In [16]:
Ks = range(2, 20)

print()
print('Start using PCA features for EM...')

print('Generate BIC bar plots...')
generate_and_plot_bic_em(Ks, X_pca, 'PCA Data', folder+'em_pca_bic.png')
print('Done...')


Start using PCA features for EM...
Generate BIC bar plots...
Done...


In [17]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_pca, 'PCA Data', folder+'em_pca_sil.png',
    xleft=-0.2, ncols=10, method='EM'
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [18]:
print('Plot EM result...')
K = 2
em_est = GaussianMixture(K, 'full', random_state=42)
y_hat = em_est.fit_predict(X_pca)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for EM on PCA: {}'.format(terrors))
covs = get_covmats(em_est.covariances_, X_pca.shape[1], K, 'full')
covmats = covs[:, 0:2, 0:2]
plot_em_2D(X_pca[:, 0], X_pca[:, 1], y, y_hat,
           em_est.means_[:, 0:2], covmats, NUM_CLASS, K,
           label_map, 'PCA Data', folder+'em_pca_result.png')
print('Done...')

print('EM on PCA data finished...')

Plot EM result...
Total error for EM on PCA: 129
Done...
EM on PCA data finished...


In [106]:
K = 29
ica = FastICA(K, random_state=42, max_iter=2000000)
X_centered = X_raw - np.mean(X_raw, axis=0)
X_centered -= np.mean(X_centered, axis=1).reshape(X.shape[0], -1)
X_ica = ica.fit_transform(X_centered)

In [107]:
fig = plt.figure(figsize=(10, 4))
ax1 = fig.add_subplot(1, 1, 1)
ax1.set_title('Kurtosis of different components')
ax1.set_xlabel('K')
ax1.set_ylabel('Kurtosis')
kur_vals = np.zeros(K, dtype=np.float)
for i in range(K):
    kur_vals[i] = kurtosis(X_ica[:, i])
ax1.plot(range(1, K+1), kur_vals, 'o-', color='blue')
plt.savefig(folder+'ica_kurtosis.png', format='png')
plt.close()

In [108]:
sel_cols = np.argsort(kur_vals)[::-1][0:2]
projs = X_ica[:, sel_cols]
plt.figure()
x_control = projs[:, 0][np.where(y == 0)]
y_control = projs[:, 1][np.where(y == 0)]
x_cancer = projs[:, 0][np.where(y == 1)]
y_cancer = projs[:, 1][np.where(y == 1)]
plt.plot(
    x_control, y_control,
    'o', ms=1, color='red', label='Control'
)
plt.plot(
    x_cancer, y_cancer,
    'o', ms=1, color='blue', label='Cancer'
)
plt.legend(loc='best')
plt.savefig(folder + 'ica_kurtosis_outlier.png', format='png')
plt.close()

In [109]:
rfc = RandomForestClassifier(100, random_state=42)
selector = RFECV(rfc, cv=5)
selector = selector.fit(X_ica, y)

X_ica = X_ica[:, selector.support_]

In [110]:
selector.support_

array([ True,  True,  True, False,  True, False,  True, False, False,
        True,  True,  True, False,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True])

In [111]:
print('Generate elbow plots...')
distortions = np.zeros(len(Ks), dtype=np.float)
for i, K in enumerate(Ks):
    distortions[i] = calculate_distortions(K, X_ica)
plot_distortions(Ks, distortions, 'ICA Data', folder+'kmeans_ica_elbow.png')
print('Done...')

Generate elbow plots...
Done...


In [112]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_ica, 'ICA Data', folder+'kmeans_ica_sil.png',
    xleft=-0.2, ncols=10
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [113]:
print('Plot k-means result...')
K = 2
km_est = KMeans(K, random_state=42)
y_hat = km_est.fit_predict(X_ica)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for KMeans on ICA: {}'.format(terrors))
plot_kmeans_2D(X_ica[:, 0], X_ica[:, 1], y, y_hat,
               km_est.cluster_centers_[:, 0:2], NUM_CLASS, K,
               label_map, 'ICA data', folder+'kmeans_ica_result.png')
print('Done...')

print('Kmeans on ICA data finished...')

Plot k-means result...
Total error for KMeans on ICA: 55
Done...
Kmeans on ICA data finished...


In [114]:
Ks = range(2, 20)

print()
print('Start using ICA features for EM...')

print('Generate BIC bar plots...')
generate_and_plot_bic_em(Ks, X_ica, 'ICA Data', folder+'em_ica_bic.png')
print('Done...')


Start using ICA features for EM...
Generate BIC bar plots...
Done...


In [115]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_ica, 'ICA Data', folder+'em_ica_sil.png',
    xleft=-0.2, ncols=10, method='EM'
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [116]:
print('Plot EM result...')
K = 2
em_est = GaussianMixture(K, 'full', random_state=42)
y_hat = em_est.fit_predict(X_ica)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for EM on ICA: {}'.format(terrors))
covs = get_covmats(em_est.covariances_, X_ica.shape[1], K, 'full')
covmats = covs[:, 0:2, 0:2]
plot_em_2D(X_ica[:, 0], X_ica[:, 1], y, y_hat,
           em_est.means_[:, 0:2], covmats, NUM_CLASS, K,
           label_map, 'ICA Data', folder+'em_ica_result.png')
print('Done...')

print('EM on ICA data finished...')

Plot EM result...
Total error for EM on ICA: 60
Done...
EM on ICA data finished...


In [51]:
print()
print('Start using RP for preprocessing...')

# Estimate the number of dimensions by JL Lemma
print(random_projection.johnson_lindenstrauss_min_dim(X.shape[0], 0.1))

K = 6
Niter = 100
terrors_rp = np.zeros(Niter, dtype=np.float)
for i, s in enumerate(range(100)):
    rp = random_projection.GaussianRandomProjection(
        K, random_state=s
    )
    X_rp = rp.fit_transform(X)
    rfc = RandomForestClassifier(
        n_estimators=100, max_depth=2,
        random_state=42
    )
    rfc.fit(X_rp, y)
    terrors_rp[i] = rfc.score(X_rp, y)


Start using RP for preprocessing...
5437


In [52]:
plt.figure()
plt.hist(terrors_rp, bins=30)
plt.savefig(folder+'rp_matcherrors_hist.png', format='png')
plt.close()

In [53]:
seed_used = np.argsort(terrors_rp)[::-1][0]
rp = random_projection.GaussianRandomProjection(
    K, random_state=seed_used
)
X_rp = rp.fit_transform(X)

In [54]:
Ks = range(2, 20)

print('Start using RP features for K-means...')

print('Generate elbow plots...')
distortions = np.zeros(len(Ks), dtype=np.float)
for i, K in enumerate(Ks):
    distortions[i] = calculate_distortions(K, X_rp)
plot_distortions(Ks, distortions, 'RP Data', folder+'kmeans_rp_elbow.png')
print('Done...')

Start using RP features for K-means...
Generate elbow plots...
Done...


In [55]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_rp, 'RP Data', folder+'kmeans_rp_sil.png',
    xleft=-0.2, ncols=10
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [56]:
print('Plot k-means result...')
K = 2
km_est = KMeans(K, random_state=42)
y_hat = km_est.fit_predict(X_rp)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for KMeans on RP: {}'.format(terrors))
plot_kmeans_2D(X_rp[:, 0], X_rp[:, 1], y, y_hat,
               km_est.cluster_centers_[:, 0:2], NUM_CLASS, K,
               label_map, 'RP data', folder+'kmeans_rp_result.png')
print('Done...')

Plot k-means result...
Total error for KMeans on RP: 43
Done...


In [57]:
Ks = range(2, 20)

print()
print('Start using RP features for EM...')

print('Generate BIC bar plots...')
generate_and_plot_bic_em(Ks, X_rp, 'RP Data', folder+'em_rp_bic.png')
print('Done...')


Start using RP features for EM...
Generate BIC bar plots...
Done...


In [58]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_rp, 'RP Data', folder+'em_rp_sil.png',
    xleft=-0.2, ncols=10, method='EM'
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [59]:
print('Plot EM result...')
K = 2
em_est = GaussianMixture(K, 'full', random_state=42)
y_hat = em_est.fit_predict(X_rp)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for EM on RP: {}'.format(terrors))
covs = get_covmats(em_est.covariances_, X_rp.shape[1], K, 'full')
covmats = covs[:, 0:2, 0:2]
plot_em_2D(X_rp[:, 0], X_rp[:, 1], y, y_hat,
           em_est.means_[:, 0:2], covmats, NUM_CLASS, K,
           label_map, 'RP Data', folder+'em_rp_result.png')
print('Done...')

print('EM on RP data finished...')

Plot EM result...
Total error for EM on RP: 34
Done...
EM on RP data finished...


In [60]:
print()

print('Start selecting number of features using RFECV...')
rfc = RandomForestClassifier(100, random_state=42)
selector = RFECV(rfc, cv=5)
selector = selector.fit(X, y)

X_fs = X[:, selector.support_]
print('Done...')


Start selecting number of features using RFECV...
Done...


In [61]:
Ks = range(2, 20)

print('Start using FS features for K-means...')

print('Generate elbow plots...')
distortions = np.zeros(len(Ks), dtype=np.float)
for i, K in enumerate(Ks):
    distortions[i] = calculate_distortions(K, X_fs)
plot_distortions(Ks, distortions, 'RFECV Data', folder+'kmeans_fs_elbow.png')
print('Done...')

Start using FS features for K-means...
Generate elbow plots...
Done...


In [62]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_fs, 'RFECV Data', folder+'kmeans_fs_sil.png',
    xleft=-0.2, ncols=10
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [63]:
print('Plot k-means result...')
K = 2
km_est = KMeans(K, random_state=42)
y_hat = km_est.fit_predict(X_fs)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total error for KMeans on RFECV: {}'.format(terrors))
plot_pca = PCA(2, random_state=42)
projs = plot_pca.fit_transform(X_fs)
plot_kmeans_2D(projs[:, 0], projs[:, 1], y, y_hat,
               plot_pca.transform(km_est.cluster_centers_), NUM_CLASS, K,
               label_map, 'RFECV data', folder+'kmeans_fs_result.png')
print('Done...')

print('Kmeans on RFECV data finished...')

Plot k-means result...
Total error for KMeans on RFECV: 43
Done...
Kmeans on RFECV data finished...


In [64]:
Ks = range(2, 20)

print()
print('Start using RFECV for EM...')

print('Generate BIC bar plots...')
generate_and_plot_bic_em(Ks, X_fs, 'RFECV Data', folder+'em_fs_bic.png')
print('Done...')


Start using RFECV for EM...
Generate BIC bar plots...
Done...


In [65]:
print('Generate Silhouette plots...')
vals = generate_and_plot_silhouette(
    Ks, X_fs, 'RFECV Data', folder+'em_fs_sil.png',
    xleft=-0.2, ncols=10, method='EM'
)
print(np.argsort(vals)[::-1][0] + 2)
print('Done...')

Generate Silhouette plots...
2
Done...


In [66]:
print('Plot EM result...')
K = 2
em_est = GaussianMixture(K, 'full', random_state=42)
y_hat = em_est.fit_predict(X_fs)
y_hat, terrors, label_map = match_labels(y, y_hat, NUM_CLASS, K)
print('Total errors for EM on RFECV: {}'.format(terrors))
plot_pca = PCA(2, random_state=42)
projs = plot_pca.fit_transform(X_fs)
covs = get_covmats(em_est.covariances_, X_fs.shape[1], K, 'full')
covmats = np.matmul(
    np.matmul(plot_pca.components_, covs),
    plot_pca.components_.T
)
plot_em_2D(projs[:, 0], projs[:, 1], y, y_hat,
           plot_pca.transform(em_est.means_), covmats, NUM_CLASS, K,
           label_map, 'RFECV Data', folder+'em_fs_result.png')
print('Done...')

print('EM on RFECV data finished...')

Plot EM result...
Total errors for EM on RFECV: 44
Done...
EM on RFECV data finished...


In [67]:
print()
print('Test neural network performances...')
nnet = MLPClassifier((10,), random_state=42, max_iter=200000)

cv_scores = np.zeros(7, dtype=np.float)
cv_stds = np.zeros(7, dtype=np.float)
run_times = np.zeros(7, dtype=np.float)

kmest = KMeans(2, random_state=42)
emest = GaussianMixture(2, 'full', random_state=42)

X_km = kmest.fit_predict(X).reshape(-1, 1)
X_em = emest.fit_predict(X).reshape(-1, 1)

features = [X, X_pca, X_ica, X_rp, X_fs, X_km, X_em]
labels = ['Raw', 'PCA', 'ICA', 'Random Projection', 'RFECV',
          'Kmeans', 'Expectation Maximum']

print('Collect data...')
for i, (X_input, name) in enumerate(zip(features, labels)):
    start = time.time()
    vals = cross_val_score(nnet, X_input, y, cv=5)
    cv_scores[i] = vals.mean()
    cv_stds[i] = vals.std()
    run_times[i] = time.time() - start

fig = plt.figure(figsize=(10, 10))
barwidth = 0.25

ax1 = fig.add_subplot(211)
midpoints = np.arange(cv_scores.shape[0])
ax1.bar(
    midpoints, cv_scores, yerr=cv_stds,
    color='red', width=barwidth, capsize=5
)
ax1.set_title('Performance using difference features')
ax1.set_xlabel('Methods')
ax1.set_ylabel('Scores')
ax1.set_xticks(midpoints)
ax1.set_xticklabels(labels)

ax2 = fig.add_subplot(212)
ax2.bar(
    midpoints, run_times,
    color='blue', width=barwidth
)
ax2.set_title('Time spent for training')
ax2.set_xlabel('Methods')
ax2.set_ylabel('Time')
ax2.set_xticks(midpoints)
ax2.set_xticklabels(labels)

plt.savefig(folder+'nn_performances.png', format='png')
plt.close()
print('Done...')


Test neural network performances...
Collect data...
Done...


In [5]:
pca = PCA(n_components = 30, random_state=42)

In [6]:
X_pca = pca.fit_transform(X)

In [7]:
em_raw = KMeans(10, random_state=42)
em_pca = KMeans(10, random_state=42)
em_raw.fit(X)
em_pca.fit(X_pca)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [8]:
y_hat_raw, _, _ = match_labels(y, em_raw.predict(X), 2, 10)
y_hat_pca, _, _ = match_labels(y, em_pca.predict(X_pca), 2, 10)

In [9]:
y_hat_raw - y_hat_pca

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
gm_raw = GaussianMixture(10, random_state=42)
gm_pca = GaussianMixture(10, random_state=42)
gm_raw.fit(X)
gm_pca.fit(X_pca)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=10, n_init=1,
                precisions_init=None, random_state=42, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [13]:
y_hat_raw, _, _ = match_labels(y, gm_raw.predict(X), 2, 10)
y_hat_pca, _, _ = match_labels(y, gm_pca.predict(X_pca), 2, 10)

In [14]:
y_hat_raw - y_hat_pca

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,